In [6]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:99%;}
div.CodeMirror {font-family:Consolas; font-size:20pt;}
div.output {font-size:18pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:19px;}
</style>
"""))

# 1. LeNet : 최초의 CNN
```
<img src='image/LeNet.png' alt='LeNet'>
```
![LeNet](image/LeNet.png)

FC(120) : 120개 출력층의 뉴런을 가진 Fully Connected 레이어

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, AvgPool2D, Flatten, Dense
                                            # AveragePooling2D 동일
from tensorflow.keras.callbacks import EarlyStopping, Callback
import pandas as pd # crosstab
from sklearn.metrics import confusion_matrix

In [3]:
class CustomHistory(Callback): # on_epoch_end()는 각 에포크 종료시 자동 호출
    def __init__(self, times=5): # 생성자 함수
        self.epoch = 0
        self.times = times
    def on_epoch_end(self, batch, logs={}):
        self.epoch += 1
        if self.epoch%self.times == 0:
            print('epoch:{}, loss:{}, acc:{}, val_loss:{}, val_acc:{}'.\
                 format(self.epoch, 
                        logs.get('loss'), 
                        logs.get('accuracy'),
                        logs.get('val_loss'),
                        logs.get('val_accuracy')
                       ))

In [5]:
# 출력사이즈 = (입력사이즈 - 커널사이즈)/스트라이드 + 1
class LeNet:
    @staticmethod
    def build(input_shape=(32,32,1), activation='relu'):
        model = Sequential()
        model.add(Input(shape=input_shape))
        model.add(Conv2D(6, # 커널 갯수
                        kernel_size=(5,5),
                        activation=activation,
                        kernel_initializer='he_normal',
                        padding='valid'
                        )) # C1층
        model.add(AvgPool2D(pool_size=(2,2),
                           strides=2, # 스트라이드 사이즈는 기본이 pool_size
                           # strides=(2,2)
                           )) # P2층
        model.add(Conv2D(16, kernel_size=(5,5),
                        activation=activation,
                        kernel_initializer='he_normal')) # C3층
        model.add(AvgPool2D(pool_size=(2,2), strides=2)) # P4층
        model.add(Flatten())
        model.add(Dense(120, activation='relu', kernel_initializer='he_normal')) #F5층
        model.add(Dense(84, activation='relu', kernel_initializer='he_normal')) # F6층
        model.add(Dense(10, activation='softmax')) # 출력층
        return model

In [7]:
class LeNet:
    @staticmethod
    def build(input_shape=(32,32,1), activation='relu'):
        model = Sequential([
            Input(shape=input_shape),
            Conv2D(6, kernel_size=(5,5), activation=activation, 
                   kernel_initializer='he_normal'),
            AvgPool2D(pool_size=(2,2), strides=2),
            Conv2D(16, kernel_size=(5,5), activation=activation, 
                   kernel_initializer='he_normal'),
            AvgPool2D(pool_size=(2,2), strides=2),
            Flatten(),
            Dense(120, activation='relu', kernel_initializer='he_normal'),
            Dense(84, activation='relu', kernel_initializer='he_normal'),
            Dense(10, activation='softmax') # 출력층
        ])
        return model

In [8]:
model = LeNet.build(input_shape=(28,28,1), activation='relu')
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d (AverageP  (None, 12, 12, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_1 (Averag  (None, 4, 4, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 120)               3

# 2. AlexNet

# 3. VGG